**Tema 4 - Metode de clasificare**

Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. `p=5%`, `p` parametru) cu `numpy.nan`. 

In [1]:
import numpy as np
import pandas as pd
import sklearn

print('At the moment the program was made:')
print('\tNumPy Version = 1.20.2')
print('\tScykit_learn Version = 0.24.1')
print('\tPandas Version = 1.2.3\n')

print(f'Current NumPy Version = {np.__version__}')
print(f'Current Sklearn Version = {sklearn.__version__}')
print(f'Current Pandas Version = {pd.__version__}')

At the moment the program was made:
	NumPy Version = 1.20.2
	Scykit_learn Version = 0.24.1
	Pandas Version = 1.2.3

Current NumPy Version = 1.20.2
Current Sklearn Version = 0.24.1
Current Pandas Version = 1.2.4


## Subpunctul 1
1. Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

### Setul de date `Planning Relax`

In [2]:
pd_data = pd.read_csv("data/planning_relax_data_set.txt", delimiter="\t", header=None)
pd_data = pd_data.drop(13, axis=1)
pd_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.17936,-0.20700,-0.209710,-0.097260,-0.119210,-0.17322,-0.280760,0.223170,0.418660,-0.032886,0.003383,-0.334250,1.0
1,-0.14659,-0.16494,0.249870,-0.073985,0.494940,-0.15633,-0.288910,0.500800,-0.045553,0.050759,-0.017511,0.066959,1.0
2,-0.13131,0.51816,0.432810,0.397890,0.043127,0.56320,-0.013246,-0.062833,-0.883390,-0.040303,0.088057,0.631200,2.0
3,0.64940,0.41878,0.148350,0.333490,-0.219740,0.39891,0.590650,0.077100,-1.107200,-0.367230,0.364680,0.571720,1.0
4,-0.40628,0.11503,0.073336,0.070066,-0.030920,0.15205,-0.191420,-0.580430,-0.102250,0.153750,-0.126950,0.184570,1.0


In [3]:
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       182 non-null    float64
 1   1       182 non-null    float64
 2   2       182 non-null    float64
 3   3       182 non-null    float64
 4   4       182 non-null    float64
 5   5       182 non-null    float64
 6   6       182 non-null    float64
 7   7       182 non-null    float64
 8   8       182 non-null    float64
 9   9       182 non-null    float64
 10  10      182 non-null    float64
 11  11      182 non-null    float64
 12  12      182 non-null    float64
dtypes: float64(13)
memory usage: 18.6 KB


In [4]:
pd_data = pd_data.values
pd_data_original = pd_data.copy()
assert not np.isnan(pd_data_original.sum()) # verificam ca nu e niciun nan 

Setul initial de date nu are valori lipsa, dar am implementat o functie care sa suprascrie un anumit procent din valorile initiale (dat ca parametru) folosind `numpy.nan`

In [8]:
def insert_nan(data, percentage=15):
    numbers_to_be_modified = int(percentage / 100 * data.shape[0] * data.shape[1])
    random_lines = np.random.choice(data.shape[0], numbers_to_be_modified)
    random_cols = np.random.choice(data.shape[1], numbers_to_be_modified)
    data[random_lines, random_cols] = np.nan
    return data

pd_data = insert_nan(pd_data)
print(f'DataSet after adding Nan: \n {pd_data}')
assert np.isnan(pd_data.sum()) # verificam ca e cel putin un nan

DataSet after adding Nan: 
 [[-0.17936   -0.207     -0.20971   ...  0.0033827 -0.33425    1.       ]
 [       nan -0.16494    0.24987   ... -0.017511   0.066959   1.       ]
 [-0.13131    0.51816    0.43281   ...  0.088057   0.6312     2.       ]
 ...
 [ 0.29877   -0.17962    0.032859  ...  0.24683   -0.38916    2.       ]
 [-0.37812   -0.52287    0.21541   ...  0.007591   0.47603    1.       ]
 [ 0.37913   -0.046562         nan ...  0.17369    0.49063    2.       ]]


In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 
# from sklearn.ensemble import ExtraTreesRegressor
imputer = IterativeImputer(max_iter=1000, random_state=0)
pd_data_sklearn = imputer.fit_transform(pd_data)
print(f'DataSet after missing values imputation: \n {pd_data_sklearn}')

DataSet after missing values imputation: 
 [[-0.17936    -0.207      -0.20971    ...  0.0033827  -0.33425
   1.        ]
 [-0.20722871 -0.16494     0.24987    ... -0.017511    0.066959
   1.        ]
 [-0.13131     0.51816     0.43281    ...  0.088057    0.6312
   2.        ]
 ...
 [ 0.29877    -0.17962     0.032859   ...  0.24683    -0.38916
   2.        ]
 [-0.37812    -0.52287     0.21541    ...  0.007591    0.47603
   1.        ]
 [ 0.37913    -0.046562   -0.0989483  ...  0.17369     0.49063
   2.        ]]


In [10]:
np.linalg.norm(pd_data_sklearn - pd_data_original)

6.133709292689028

### Setul de date `Conventional Social Media Movies`

In [19]:
data = pd.read_excel('data/2014 and 2015 CSM dataset.xlsx', index_col=None, header=None)
data = data.drop(0, axis=0)
data = data.drop(0, axis=1)

In [20]:
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
1,2014,6.3,8,9130,4000000,45,1,0,3280543,4632,425,636,1120000
2,2014,7.1,1,192000000,50000000,3306,2,2,583289,3465,61,186,12350000
3,2014,6.2,1,30700000,28000000,2872,1,0,304861,328,34,47,483000
4,2014,6.3,1,106000000,110000000,3470,2,0,452917,2429,132,590,568000
5,2014,4.7,8,17300000,3500000,2310,2,0,3145573,12163,610,1082,1923800


In [21]:
data = data.values
print(data)
data_original = data.copy()
assert np.isnan(data.sum()) # verificam ca e cel putin un nan

[[2014 6.3 8 ... 425 636 1120000]
 [2014 7.1 1 ... 61 186 12350000]
 [2014 6.2 1 ... 34 47 483000]
 ...
 [2015 5.4 8 ... 325 409 nan]
 [2015 5.4 1 ... 67 201 nan]
 [2015 4.4 15 ... 431 606 nan]]


In [22]:
!pip install missingpy

In [23]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [24]:
from missingpy import MissForest

imputer = MissForest()
data = imputer.fit_transform(data)

Iteration: 0
Iteration: 1
Iteration: 2


In [25]:
print(f'Data original:\n {data_original}')
print(f'\nData filled:\n {data}')

assert not np.isnan(data.sum()) # verificam ca nu exista niciun nan

Data original:
 [[2014 6.3 8 ... 425 636 1120000]
 [2014 7.1 1 ... 61 186 12350000]
 [2014 6.2 1 ... 34 47 483000]
 ...
 [2015 5.4 8 ... 325 409 nan]
 [2015 5.4 1 ... 67 201 nan]
 [2015 4.4 15 ... 431 606 nan]]

Data filled:
 [[2.01400000e+03 6.30000000e+00 8.00000000e+00 ... 4.25000000e+02
  6.36000000e+02 1.12000000e+06]
 [2.01400000e+03 7.10000000e+00 1.00000000e+00 ... 6.10000000e+01
  1.86000000e+02 1.23500000e+07]
 [2.01400000e+03 6.20000000e+00 1.00000000e+00 ... 3.40000000e+01
  4.70000000e+01 4.83000000e+05]
 ...
 [2.01500000e+03 5.40000000e+00 8.00000000e+00 ... 3.25000000e+02
  4.09000000e+02 2.07559865e+06]
 [2.01500000e+03 5.40000000e+00 1.00000000e+00 ... 6.70000000e+01
  2.01000000e+02 8.02575590e+05]
 [2.01500000e+03 4.40000000e+00 1.50000000e+01 ... 4.31000000e+02
  6.06000000e+02 2.63669288e+06]]


## Subpunctul 2
2. Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, precision, recall, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics), [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor. 

### Setul de date `Planning Relax`

In [26]:
pr_data = pd.read_csv("data/planning_relax_data_set.txt", delimiter="\t", header=None)
pr_data = pr_data.drop(13, axis=1)
pr_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.17936,-0.20700,-0.209710,-0.097260,-0.119210,-0.17322,-0.280760,0.223170,0.418660,-0.032886,0.003383,-0.334250,1.0
1,-0.14659,-0.16494,0.249870,-0.073985,0.494940,-0.15633,-0.288910,0.500800,-0.045553,0.050759,-0.017511,0.066959,1.0
2,-0.13131,0.51816,0.432810,0.397890,0.043127,0.56320,-0.013246,-0.062833,-0.883390,-0.040303,0.088057,0.631200,2.0
3,0.64940,0.41878,0.148350,0.333490,-0.219740,0.39891,0.590650,0.077100,-1.107200,-0.367230,0.364680,0.571720,1.0
4,-0.40628,0.11503,0.073336,0.070066,-0.030920,0.15205,-0.191420,-0.580430,-0.102250,0.153750,-0.126950,0.184570,1.0


In [27]:
pr_X = pr_data.iloc[:, :-1].values
pr_y = pr_data.iloc[:, -1].values

In [28]:
print(pr_X)
print(pr_y)

[[-0.17936   -0.207     -0.20971   ... -0.032886   0.0033827 -0.33425  ]
 [-0.14659   -0.16494    0.24987   ...  0.050759  -0.017511   0.066959 ]
 [-0.13131    0.51816    0.43281   ... -0.040303   0.088057   0.6312   ]
 ...
 [ 0.29877   -0.17962    0.032859  ... -0.41113    0.24683   -0.38916  ]
 [-0.37812   -0.52287    0.21541   ... -0.039117   0.007591   0.47603  ]
 [ 0.37913   -0.046562  -0.12336   ... -0.24229    0.17369    0.49063  ]]
[1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2.
 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2.]


In [29]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn import metrics
  
def calculate_scores_micro(model, X, y):
    scores = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'precision', 'recall', 'f1'], return_train_score=True)
    train_accuracy = scores['train_accuracy'].mean()
    test_accuracy = scores['test_accuracy'].mean()
    train_precision = scores['train_precision'].mean()
    test_precision = scores['test_precision'].mean()
    train_recall = scores['train_recall'].mean()
    test_recall = scores['test_recall'].mean()
    train_f1_score = scores['train_f1'].mean()
    test_f1_score = scores['test_f1'].mean()
    print(f'Train accuracy: {train_accuracy}')
    print(f'Test accuracy: {test_accuracy}')
    print(f'Train precision: {train_precision}')
    print(f'Test precision: {test_precision}')
    print(f'Train recall: {train_recall}')
    print(f'Test recall: {test_recall}')
    print(f'Train F1: {train_f1_score}')
    print(f'Test F1: {test_f1_score}')

* Folosind `KNeighborsClassifier`

In [30]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
calculate_scores_micro(model=model, X=pr_X, y=pr_y)

Train accuracy: 0.8186679263108172
Test accuracy: 0.6376876876876877
Train precision: 0.8283701291123394
Test precision: 0.7074860354699064
Train recall: 0.9423076923076923
Test recall: 0.8384615384615384
Train F1: 0.8814538535510122
Test F1: 0.7668308784169008


* Folosind `DecisionTreeClassifier`

In [31]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
calculate_scores_micro(model=classifier, X=pr_X, y=pr_y)

Train accuracy: 1.0
Test accuracy: 0.5615615615615617
Train precision: 1.0
Test precision: 0.6856716718785685
Train recall: 1.0
Test recall: 0.7
Train F1: 1.0
Test F1: 0.6912879643068324


* Folosind `SupportVectorMachines`

In [32]:
from sklearn.svm import SVC

classifier = SVC(kernel='linear')
calculate_scores_micro(model=classifier, X=pr_X, y=pr_y)

Train accuracy: 0.7142938119981105
Test accuracy: 0.7144144144144144
Train precision: 0.7142938119981105
Test precision: 0.7144144144144144
Train recall: 1.0
Test recall: 1.0
Train F1: 0.833336546184739
Test F1: 0.8333845366103431


* Folosind `SochasticGradientDescent`

In [33]:
from sklearn.linear_model import SGDClassifier

classifier=SGDClassifier(loss='hinge', penalty='l2')
calculate_scores_micro(model=classifier, X=pr_X, y=pr_y)

Train accuracy: 0.6745772319319792
Test accuracy: 0.570870870870871
Train precision: 0.7331567287612207
Test precision: 0.6761511022871471
Train recall: 0.8557692307692306
Test recall: 0.7615384615384616
Train F1: 0.787393859050052
Test F1: 0.7112899185207492


* Folosind `RBF`

In [34]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel)
calculate_scores_micro(model=gpc, X=pr_X, y=pr_y)

Train accuracy: 0.7142938119981105
Test accuracy: 0.7144144144144144
Train precision: 0.7142938119981105
Test precision: 0.7144144144144144
Train recall: 1.0
Test recall: 1.0
Train F1: 0.833336546184739
Test F1: 0.8333845366103431


### Setul de date `Divorce`


In [35]:
divorce_data = pd.read_excel('data/divorce.xlsx', index_col=None, header=None)
divorce_data = divorce_data.drop(0, axis=0)

In [36]:
# divorce_data.head()
print(divorce_data)

    0  1  2  3  4  5  6  7  8  9   ... 45 46 47 48 49 50 51 52 53 54
1    2  2  4  1  0  0  0  0  0  0  ...  2  1  3  3  3  2  3  2  1  1
2    4  4  4  4  4  0  0  4  4  4  ...  2  2  3  4  4  4  4  2  2  1
3    2  2  2  2  1  3  2  1  1  2  ...  3  2  3  1  1  1  2  2  2  1
4    3  2  3  2  3  3  3  3  3  3  ...  2  2  3  3  3  3  2  2  2  1
5    2  2  1  1  1  1  0  0  0  0  ...  2  1  2  3  2  2  2  1  0  1
..  .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
166  0  0  0  0  0  0  0  0  0  0  ...  1  0  4  1  1  4  2  2  2  0
167  0  0  0  0  0  0  0  0  0  0  ...  4  1  2  2  2  2  3  2  2  0
168  1  1  0  0  0  0  0  0  0  1  ...  3  0  2  0  1  1  3  0  0  0
169  0  0  0  0  0  0  0  0  0  0  ...  3  3  2  2  3  2  4  3  1  0
170  0  0  0  0  0  0  0  1  0  0  ...  3  4  4  0  1  3  3  3  1  0

[170 rows x 55 columns]


In [37]:
divorce_X = divorce_data.iloc[:, :-1].values
divorce_y = divorce_data.iloc[:, -1].values
divorce_y = divorce_y.astype('int')

* Folosind `KNeighborsClassifier`

In [38]:
#from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
calculate_scores_micro(model=model, X=divorce_X, y=divorce_y)

Train accuracy: 0.9779411764705882
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 1.0
Train recall: 0.9553555750658473
Test recall: 0.9529411764705882
Train F1: 0.9770271833429728
Test F1: 0.9733333333333334


* Folosind `DecisionTreeClassifier`

In [39]:
classifier = DecisionTreeClassifier()
calculate_scores_micro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9705882352941175
Train precision: 1.0
Test precision: 0.9777777777777779
Train recall: 1.0
Test recall: 0.9647058823529411
Train F1: 1.0
Test F1: 0.9688804554079697


* Folosind `SupportVectorMachines`

In [40]:
classifier = SVC(kernel='linear')
calculate_scores_micro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 1.0
Train recall: 1.0
Test recall: 0.9529411764705882
Train F1: 1.0
Test F1: 0.9733333333333334


* Folosind `SochasticGradientDescent`

In [41]:
classifier=SGDClassifier(loss='hinge', penalty='l2')
calculate_scores_micro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9588235294117646
Train precision: 1.0
Test precision: 0.9582352941176471
Train recall: 1.0
Test recall: 0.9647058823529411
Train F1: 1.0
Test F1: 0.9583683390135004


* Folosind `RBF`

In [42]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel)
calculate_scores_micro(model=gpc, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 1.0
Train recall: 1.0
Test recall: 0.9529411764705882
Train F1: 1.0
Test F1: 0.9733333333333334


### Setul de date `HCV`

In [43]:
data_HCV = pd.read_csv('data/hcv_data.csv')
data_HCV = data_HCV.drop('Unnamed: 0', axis=1)
data_HCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    object 
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    object 
 3   ALB       614 non-null    float64
 4   ALP       597 non-null    float64
 5   ALT       614 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      605 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      614 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 62.6+ KB


In [44]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

data_HCV.Category = data_HCV.Category.astype('category')
data_HCV.Category = data_HCV.Category.cat.codes
data_HCV.Sex = data_HCV.Sex.astype('category')
data_HCV.Sex = data_HCV.Sex.cat.codes
data_HCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    int8   
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    int8   
 3   ALB       614 non-null    float64
 4   ALP       597 non-null    float64
 5   ALT       614 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      605 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      614 non-null    float64
dtypes: float64(10), int64(1), int8(2)
memory usage: 54.2 KB


In [45]:
data_HCV = data_HCV.fillna(data_HCV.mean())
data_HCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    int8   
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    int8   
 3   ALB       615 non-null    float64
 4   ALP       615 non-null    float64
 5   ALT       615 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      615 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      615 non-null    float64
dtypes: float64(10), int64(1), int8(2)
memory usage: 54.2 KB


In [46]:
HCV_X = data_HCV.iloc[:, 1:].values
HCV_y = data_HCV.iloc[:, 0].values

In [47]:
np.any(np.isnan(data_HCV))

False

In [48]:
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [49]:
def calculate_scores_macro(model, X, y):
    scores = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=True)
    train_accuracy = scores['train_accuracy'].mean()
    test_accuracy = scores['test_accuracy'].mean()
    train_precision = scores['train_precision_macro'].mean()
    test_precision = scores['test_precision_macro'].mean()
    train_recall = scores['train_recall_macro'].mean()
    test_recall = scores['test_recall_macro'].mean()
    train_f1_score = scores['train_f1_macro'].mean()
    test_f1_score = scores['test_f1_macro'].mean()
    print(f'Train accuracy: {train_accuracy}')
    print(f'Test accuracy: {test_accuracy}')
    print(f'Train precision: {train_precision}')
    print(f'Test precision: {test_precision}')
    print(f'Train recall: {train_recall}')
    print(f'Test recall: {test_recall}')
    print(f'Train F1: {train_f1_score}')
    print(f'Test F1: {test_f1_score}')

* Folosind `KNeighborsClassifier`

In [50]:
model = KNeighborsClassifier(n_neighbors=3)
calculate_scores_macro(model=model, X=HCV_X, y=HCV_y)

Train accuracy: 0.9443089430894309
Test accuracy: 0.9008130081300812
Train precision: 0.8095025426719882
Test precision: 0.524628733997155
Train recall: 0.6235858862253356
Test recall: 0.39649409275260095
Train F1: 0.6855780932171947
Test F1: 0.4394494925234267


* Folosind `DecisionTreeClassifier`

In [51]:
classifier = DecisionTreeClassifier()
calculate_scores_macro(model=classifier, X=HCV_X, y=HCV_y)

Train accuracy: 1.0
Test accuracy: 0.8878048780487806
Train precision: 1.0
Test precision: 0.48372573913368927
Train recall: 1.0
Test recall: 0.5243261035678599
Train F1: 1.0
Test F1: 0.49524932982243586


* Folosind `SupportVectorMachines`

In [52]:
classifier = SVC(kernel='linear')
calculate_scores_macro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 0.980952380952381
Train recall: 1.0
Test recall: 0.9764705882352942
Train F1: 1.0
Test F1: 0.9761403508771931


* Folosind `SochasticGradientDescent`

In [53]:
classifier=SGDClassifier(loss='hinge', penalty='l2')
calculate_scores_macro(model=classifier, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9647058823529411
Train precision: 1.0
Test precision: 0.9695144724556488
Train recall: 1.0
Test recall: 0.965032679738562
Train F1: 1.0
Test F1: 0.9643654591023012


* Folosind `RBF`

In [54]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel)
calculate_scores_macro(model=gpc, X=divorce_X, y=divorce_y)

Train accuracy: 1.0
Test accuracy: 0.9764705882352942
Train precision: 1.0
Test precision: 0.980952380952381
Train recall: 1.0
Test recall: 0.9764705882352942
Train F1: 1.0
Test F1: 0.9761403508771931


### Setul de date `Conventional Social Media Movies`

In [55]:
CSM_data = pd.read_excel('data/2014 and 2015 CSM dataset.xlsx', index_col=None, header=None)
CSM_data = CSM_data.drop(0, axis=0)
CSM_data = CSM_data.drop(0, axis=1)

In [56]:
CSM_data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
1,2014,6.3,8,9130,4000000,45,1,0,3280543,4632,425,636,1120000
2,2014,7.1,1,192000000,50000000,3306,2,2,583289,3465,61,186,12350000
3,2014,6.2,1,30700000,28000000,2872,1,0,304861,328,34,47,483000
4,2014,6.3,1,106000000,110000000,3470,2,0,452917,2429,132,590,568000
5,2014,4.7,8,17300000,3500000,2310,2,0,3145573,12163,610,1082,1923800


In [57]:
from missingpy import MissForest

imputer = MissForest()
CSM_data = imputer.fit_transform(CSM_data)
assert not np.isnan(CSM_data.sum()) # verificam ca nu exista niciun nan

Iteration: 0
Iteration: 1
Iteration: 2


In [58]:
CSM_y = CSM_data[:, 2]
CSM_X = CSM_data[:, [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

Datorita diferentelor foarte mari de valori din tabel, am decis sa facem o scalare a valorilor, pentru o clasificare mai buna.

In [59]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(CSM_X)
CSM_X = scaler.transform(CSM_X)

print(CSM_X)

[[0.00000000e+00 5.71428571e-01 1.03577381e-05 ... 3.04441261e-02
  1.65784741e-02 3.60609875e-02]
 [0.00000000e+00 7.14285714e-01 2.98597617e-01 ... 4.36962751e-03
  4.84842166e-03 3.97981252e-01]
 [0.00000000e+00 5.53571429e-01 4.77412876e-02 ... 2.43553009e-03
  1.22513881e-03 1.55317614e-02]
 ...
 [1.00000000e+00 4.10714286e-01 1.58593611e-02 ... 2.32808023e-02
  1.06613143e-02 6.46230728e-02]
 [1.00000000e+00 4.10714286e-01 1.91253145e-02 ... 4.79942693e-03
  5.23942340e-03 2.61706039e-02]
 [1.00000000e+00 2.32142857e-01 3.51440386e-02 ... 3.08739255e-02
  1.57964706e-02 7.93763808e-02]]


In [60]:
from sklearn.model_selection import ShuffleSplit
def our_function_macro(model, X, y):
    cv = ShuffleSplit(n_splits=10, test_size=1/3, random_state=0)
    scores = cross_validate(model, X, y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=True)
    train_accuracy = scores['train_accuracy'].mean()
    test_accuracy = scores['test_accuracy'].mean()
    train_precision = scores['train_precision_macro'].mean()
    test_precision = scores['test_precision_macro'].mean()
    train_recall = scores['train_recall_macro'].mean()
    test_recall = scores['test_recall_macro'].mean()
    train_f1_score = scores['train_f1_macro'].mean()
    test_f1_score = scores['test_f1_macro'].mean()
    print(f'Train accuracy: {train_accuracy}')
    print(f'Test accuracy: {test_accuracy}')
    print(f'Train precision: {train_precision}')
    print(f'Test precision: {test_precision}')
    print(f'Train recall: {train_recall}')
    print(f'Test recall: {test_recall}')
    print(f'Train F1: {train_f1_score}')
    print(f'Test F1: {test_f1_score}')

* Folosind `KNeighborsClassifier`

In [61]:
model = KNeighborsClassifier(n_neighbors=5)
# our_function_macro(model=model, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=model, X=CSM_X, y=CSM_y)

Train accuracy: 0.5097414806110457
Test accuracy: 0.24625346901017578
Train precision: 0.39475357135152184
Test precision: 0.07329378958706512
Train recall: 0.2561300171902709
Test recall: 0.10624363291029956
Train F1: 0.25639528387079125
Test F1: 0.08214560645018162


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `DecisionTreeClassifier`

In [62]:
classifier = DecisionTreeClassifier()
# our_function_macro(model=classifier, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=classifier, X=CSM_X, y=CSM_y)

Train accuracy: 1.0
Test accuracy: 0.21618871415356153
Train precision: 1.0
Test precision: 0.11273689922812728
Train recall: 1.0
Test recall: 0.13145730812397477
Train F1: 1.0
Test F1: 0.11633435373841738


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `SupportVectorMachines`

In [63]:
classifier = SVC()
# our_function_macro(model=classifier, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=classifier, X=CSM_X, y=CSM_y)

Train accuracy: 0.4274853113983549
Test accuracy: 0.346068455134135
Train precision: 0.119735259740762
Test precision: 0.11725339919847928
Train recall: 0.16509945641023865
Test recall: 0.15703358370025036
Train F1: 0.13483556536510283
Test F1: 0.11100186587259989


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `SochasticGradientDescent`

In [64]:
classifier=SGDClassifier(loss='hinge', penalty='l2')
# our_function_macro(model=classifier, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=classifier, X=CSM_X, y=CSM_y)

Train accuracy: 0.33981198589894246
Test accuracy: 0.2639222941720629
Train precision: 0.23820139569532492
Test precision: 0.09633051087398914
Train recall: 0.2192612297834285
Test recall: 0.1303824570491237
Train F1: 0.18025050526314007
Test F1: 0.08858378591973569


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


* Folosind `RBF`

In [65]:
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e10))
gpc = GaussianProcessClassifier(kernel=kernel)
# our_function_macro(model=gpc, X=CSM_X, y=CSM_y)
calculate_scores_macro(model=gpc, X=CSM_X, y=CSM_y)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Train accuracy: 0.4296122209165688
Test accuracy: 0.3548566142460684
Train precision: 0.19290913952430758
Test precision: 0.13321297314847366
Train recall: 0.1967449574532027
Test recall: 0.17956660623327292
Train F1: 0.18034240388781547
Test F1: 0.1375211010093026


## Subpunctul 3
3. Raportati performanta fiecarui model, folosind 5 fold cross validation. Pentru fiecare din cele 5 rulari, cautati hiperparametrii optimi folosind 4-fold cross validation. Performanta modelului va fi raportata ca medie a celor  5 rulari. 
    
    *Observatie:* la fiecare din cele 5 rulari, hiperparametrii optimi pot diferi, din cauza datelor utilizate pentru antrenare/validare. 

### Setul de date `Planning Relax`

* Folosind `KNeighborsClassifier`

In [60]:
def mean_score_for_model_K(k:int, X, y) -> float:
    """Creates and train a KNeighborclassifier with number of neighbors given by :param k:
    :param k: number of neighbors
    :return: mean of scores over 10 fold CV
    """
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

range_k = range(1, 35)


In [61]:
pr_data = pd.read_csv("data/planning_relax_data_set.txt", delimiter="\t", header=None)
pr_data = pr_data.drop(13, axis=1)
pr_X = pr_data.iloc[:, :-1].values
pr_y = pr_data.iloc[:, -1].values

In [63]:
from sklearn.model_selection import cross_validate

scores_k = [mean_score_for_model_K(k, pr_X, pr_y) for k in range_k]
hyp_param = 1+np.argmax(scores_k)

model = KNeighborsClassifier(n_neighbors=hyp_param)

results = cross_validate(model, pr_X, pr_y, cv=5, scoring='accuracy', return_train_score=True)
print(results)

{'fit_time': array([0.00099778, 0.00099421, 0.        , 0.0009985 , 0.        ]), 'score_time': array([0.00099659, 0.00099921, 0.00199485, 0.0009973 , 0.00199318]), 'test_score': array([0.7027027 , 0.7027027 , 0.72222222, 0.72222222, 0.72222222]), 'train_score': array([0.70344828, 0.71724138, 0.71232877, 0.69178082, 0.70547945])}


* Folosind `DecisionTreeClassifier`

In [ ]:
def mean_score_for_model_DTC(k:int, X, y) -> float:
    """Creates and train a KNeighborclassifier with number of neighbors given by :param k:
    :param k: number of neighbors
    :return: mean of scores over 10 fold CV
    """
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

In [ ]:

our_function_macro(model=classifier, X=CSM_X, y=CSM_y)

scores_k = [mean_score_for_model_K(k, pr_X, pr_y) for k in range_k]
hyp_param = 1+np.argmax(scores_k)

model = DecisionTreeClassifier(n_neighbors=hyp_param)

results = cross_validate(model, pr_X, pr_y, cv=hyp_param, scoring='accuracy', return_train_score=True)
print(results)

In [168]:
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import accuracy_score

medii = []

for _ in range(5):
    best_score = 0
    for k in range(1, 31):
        for p in [1, 2, 3, 10]:
            model = KNeighborsClassifier(n_neighbors=k, p=p)
            score = np.mean(cross_val_score(model, pr_X, pr_y, cv=10))
            if score >= best_score:
                best_score = score
                best_params = {'n_neighbors':k, 'p':p}

    print('\nIteration ', _)
    print('\tBest score:', best_score)
    print('\tBest params:', best_params)  

    model = KNeighborsClassifier(**best_params)
    model.fit(pr_X, pr_y)
    y_predicted = model.predict(pr_X)
    print('\tAccuracy on whole set:', accuracy_score(pr_y, y_predicted))
    medii += [accuracy_score(pr_y, y_predicted)]

print(f'Media acuratetei dupa 5 iteratii este: {np.mean(medii)}')


Iteration  0
	Best score: 0.725438596491228
	Best params: {'n_neighbors': 2, 'p': 3}
	Accuracy on whole set: 0.8021978021978022

Iteration  1
	Best score: 0.725438596491228
	Best params: {'n_neighbors': 2, 'p': 3}
	Accuracy on whole set: 0.8021978021978022

Iteration  2
	Best score: 0.725438596491228
	Best params: {'n_neighbors': 2, 'p': 3}
	Accuracy on whole set: 0.8021978021978022

Iteration  3
	Best score: 0.725438596491228
	Best params: {'n_neighbors': 2, 'p': 3}
	Accuracy on whole set: 0.8021978021978022

Iteration  4
	Best score: 0.725438596491228
	Best params: {'n_neighbors': 2, 'p': 3}
	Accuracy on whole set: 0.8021978021978022
Media acuratetei dupa 5 iteratii este: 0.8021978021978022
